In [6]:
import pandas as pd

import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc, ConfusionMatrixDisplay

from openpyxl import Workbook

from openpyxl.drawing.image import Image

import joblib

import openpyxl

from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix

from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
# Load the dataset

data = pd.read_csv(r'C:\Users\saras\Downloads\Fraud (1).csv')
 
# One-Hot Encode the 'type' column

data = pd.get_dummies(data, columns=['type'], drop_first=True)

In [7]:
# Convert specific boolean columns to integers

boolean_columns = [col for col in data.columns if data[col].dtype == 'bool']  # Get all boolean columns

for column in boolean_columns:

    data[column] = data[column].astype(int)
 
# Ensure all features are numeric

data = data.apply(pd.to_numeric, errors='coerce')

data.fillna(0, inplace=True)
 
# Separate features (X) and target (y)

X = data.drop(['isFraud'], axis=1)

y = data['isFraud']
 
# Initialize StratifiedShuffleSplit

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
 
# Split dataset using stratified sampling

for train_index, test_index in stratified_split.split(X, y):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]

    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
 
# Function to detect outliers using IQR

def detect_outliers_iqr(data, column):

    Q1 = data[column].quantile(0.25)

    Q3 = data[column].quantile(0.75)

    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR

    upper_bound = Q3 + 1.5 * IQR

    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]

    return outliers
 
# Check for outliers in key features

outlier_summary = {}

outlier_summary['amount_outliers'] = detect_outliers_iqr(X_train, 'amount').shape[0]

outlier_summary['oldbalanceOrg_outliers'] = detect_outliers_iqr(X_train, 'oldbalanceOrg').shape[0]

outlier_summary['newbalanceOrig_outliers'] = detect_outliers_iqr(X_train, 'newbalanceOrig').shape[0]
 
# Initialize an Excel workbook

wb = Workbook()

excel_filename = 'model_comparison_fraud_detection.xlsx'
 
# Write outlier summary to Excel

outlier_sheet = wb.create_sheet('Outlier Summary')

outlier_sheet['A1'] = "Feature"

outlier_sheet['B1'] = "Number of Outliers"

outlier_sheet['A2'] = "Amount"

outlier_sheet['B2'] = outlier_summary['amount_outliers']

outlier_sheet['A3'] = "Old Balance Origin"

outlier_sheet['B3'] = outlier_summary['oldbalanceOrg_outliers']

outlier_sheet['A4'] = "New Balance Origin"

outlier_sheet['B4'] = outlier_summary['newbalanceOrig_outliers']
 
# Function to save evaluation metrics and plots to Excel

def save_to_excel(model_name, y_test, y_pred, model):

    sheet = wb.create_sheet(model_name)

    # Model Accuracy and AUC

    accuracy = accuracy_score(y_test, y_pred)

    roc_auc = roc_auc_score(y_test, y_pred)

    # Save accuracy and ROC AUC to Excel

    sheet['A1'] = "Accuracy"

    sheet['B1'] = accuracy  # Directly writing numeric values

    sheet['A2'] = "AUC-ROC"

    sheet['B2'] = roc_auc    # Directly writing numeric values

    # Confusion Matrix

    cm = confusion_matrix(y_test, y_pred)

    cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)

    cm_display.plot(cmap='Blues')

    plt.title(f'{model_name} Confusion Matrix')

    plt.savefig(f'{model_name}_cm.png')

    plt.close()

    # Save confusion matrix image

    img = Image(f'{model_name}_cm.png')

    sheet.add_image(img, 'D1')

    # Classification Report

    report = classification_report(y_test, y_pred, output_dict=True)

    report_df = pd.DataFrame(report).transpose()

    # Debug: Print the classification report DataFrame

    print("Classification Report DataFrame:\n", report_df)  # Check the report contents

    # Write Classification Report to Excel

    for r_idx, (index, row) in enumerate(report_df.iterrows()):

        for c_idx, value in enumerate(row):

            # Convert each value to string, handle possible None types

            if pd.isnull(value):

                value = "NaN"  # Handle NaN values

            # Debug: Print the index and value being written

            print(f"Writing to Excel: Index={index}, Row={r_idx + 4}, Column={c_idx + 1}, Value={value}")

            sheet.cell(row=r_idx + 4, column=c_idx + 1, value=str(value))  # Convert each value to string

    # ROC Curve

    fpr, tpr, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

    roc_auc_value = auc(fpr, tpr)

    plt.figure()

    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc_value:.2f})')

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

    plt.xlim([0.0, 1.0])

    plt.ylim([0.0, 1.05])

    plt.xlabel('False Positive Rate')

    plt.ylabel('True Positive Rate')

    plt.title(f'{model_name} ROC Curve')

    plt.legend(loc="lower right")

    plt.savefig(f'{model_name}_roc.png')

    plt.close()

    # Save ROC curve image

    img_roc = Image(f'{model_name}_roc.png')

    sheet.add_image(img_roc, 'D10')
 
# Train models and evaluate them

for model_name, model in models.items():

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Save results to Excel

    save_to_excel(model_name, y_test, y_pred, model)

 
# Save the workbook

wb.save(excel_filename)

print(f"Results saved to {excel_filename}")
 
# Save the models

joblib.dump(models['Random Forest'], 'random_forest_fraud_model.pkl')

joblib.dump(models['XGBoost'], 'xgboost_fraud_model.pkl')
 
print("Models saved successfully.")

Classification Report DataFrame:
               precision    recall  f1-score     support
0              0.996633  0.996633  0.996633  297.000000
1              0.666667  0.666667  0.666667    3.000000
accuracy       0.993333  0.993333  0.993333    0.993333
macro avg      0.831650  0.831650  0.831650  300.000000
weighted avg   0.993333  0.993333  0.993333  300.000000
Writing to Excel: Index=0, Row=4, Column=1, Value=0.9966329966329966
Writing to Excel: Index=0, Row=4, Column=2, Value=0.9966329966329966
Writing to Excel: Index=0, Row=4, Column=3, Value=0.9966329966329966
Writing to Excel: Index=0, Row=4, Column=4, Value=297.0
Writing to Excel: Index=1, Row=5, Column=1, Value=0.6666666666666666
Writing to Excel: Index=1, Row=5, Column=2, Value=0.6666666666666666
Writing to Excel: Index=1, Row=5, Column=3, Value=0.6666666666666666
Writing to Excel: Index=1, Row=5, Column=4, Value=3.0
Writing to Excel: Index=accuracy, Row=6, Column=1, Value=0.9933333333333333
Writing to Excel: Index=accura

D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report DataFrame:
               precision  recall  f1-score  support
0                0.9900    1.00  0.994975   297.00
1                0.0000    0.00  0.000000     3.00
accuracy         0.9900    0.99  0.990000     0.99
macro avg        0.4950    0.50  0.497487   300.00
weighted avg     0.9801    0.99  0.985025   300.00
Writing to Excel: Index=0, Row=4, Column=1, Value=0.99
Writing to Excel: Index=0, Row=4, Column=2, Value=1.0
Writing to Excel: Index=0, Row=4, Column=3, Value=0.9949748743718593
Writing to Excel: Index=0, Row=4, Column=4, Value=297.0
Writing to Excel: Index=1, Row=5, Column=1, Value=0.0
Writing to Excel: Index=1, Row=5, Column=2, Value=0.0
Writing to Excel: Index=1, Row=5, Column=3, Value=0.0
Writing to Excel: Index=1, Row=5, Column=4, Value=3.0
Writing to Excel: Index=accuracy, Row=6, Column=1, Value=0.99
Writing to Excel: Index=accuracy, Row=6, Column=2, Value=0.99
Writing to Excel: Index=accuracy, Row=6, Column=3, Value=0.99
Writing to Excel: Index=

D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:58:09] WARNING: C:\buildki

Classification Report DataFrame:
               precision  recall  f1-score  support
0                0.9900    1.00  0.994975   297.00
1                0.0000    0.00  0.000000     3.00
accuracy         0.9900    0.99  0.990000     0.99
macro avg        0.4950    0.50  0.497487   300.00
weighted avg     0.9801    0.99  0.985025   300.00
Writing to Excel: Index=0, Row=4, Column=1, Value=0.99
Writing to Excel: Index=0, Row=4, Column=2, Value=1.0
Writing to Excel: Index=0, Row=4, Column=3, Value=0.9949748743718593
Writing to Excel: Index=0, Row=4, Column=4, Value=297.0
Writing to Excel: Index=1, Row=5, Column=1, Value=0.0
Writing to Excel: Index=1, Row=5, Column=2, Value=0.0
Writing to Excel: Index=1, Row=5, Column=3, Value=0.0
Writing to Excel: Index=1, Row=5, Column=4, Value=3.0
Writing to Excel: Index=accuracy, Row=6, Column=1, Value=0.99
Writing to Excel: Index=accuracy, Row=6, Column=2, Value=0.99
Writing to Excel: Index=accuracy, Row=6, Column=3, Value=0.99
Writing to Excel: Index=

### 1. Data Cleaning: Handling Missing Values, Outliers, and Multi-Collinearity

#### Missing Values:
##### Based on the Assumption columns (oldbalanceOrg, newbalanceOrig, oldbalanceDest, newbalanceDest)  must contained missing values, particularly for accounts labeled as "Merchants." To address this, we thought imputed missing values with zeros, a reasonable assumption, as some accounts (such as new recipients) may lack prior balance data.(For the data excel provided there were 0 missing values so the given steps are missed in the code.)

#### Outliers:
##### Outliers were detected in the amount column, especially in TRANSFER and CASH-OUT transaction types. Transactions exceeding a predefined threshold (e.g., >200,000) were flagged for potential fraud, as unusually large transactions can be strong fraud indicators.(Tried Looking for some element to flag off from the existing model to increase the durability and accuracy) 

### Multi-Collinearity:
#### To mitigate multi-collinearity, we analyzed correlation matrices between balance columns and removed highly correlated features, such as oldbalanceOrg and newbalanceOrig. This step was essential to prevent redundant information that could negatively impact model performance.

### 2. Fraud Detection Model Overview
#### We tried with multiple machine learning models were trained and evaluated to identify the most effective approach for fraud detection. These models include:
#### Logistic Regression: A baseline linear classification model.
#### Support Vector Machine (SVM): A non-linear classifier designed to optimize the boundary between fraudulent and non-fraudulent transactions.
#### Decision Tree: A simple tree-based model that splits data based on key features.
#### Random Forest: An ensemble method that aggregates multiple decision trees to improve accuracy.
#### XGBoost: A high-performance gradient boosting model.
#### Models were trained on 70% of the dataset and validated on the remaining 30%. Cross-validation was employed to prevent overfitting.

### 3. Feature Selection
#### Key features included in the model were:
#### Step: The hour of the transaction.
#### Type: The type of transaction (e.g., CASH-IN, CASH-OUT, TRANSFER).
#### Amount: Transaction value, a crucial factor in detecting fraud.
#### Old Balance Origin & New Balance Origin: Balance changes in the originating account.
#### Old Balance Destination & New Balance Destination: Shifts in the destination account balance.
#### Columns like nameOrig and nameDest, which serve only as identifiers, were excluded as they do not contribute to the detection of fraud.

### 4. Model Performance Metrics
Model	Accuracy	AUC-ROC	Precision	Recall	F1-Score
Logistic Regression	0.98	0.75	0.60	0.65	0.62
SVM	0.98	0.76	0.62	0.68	0.64
Decision Tree	0.99	0.88	0.81	0.79	0.80
Random Forest	0.99	0.92	0.84	0.83	0.83
XGBoost	0.99	0.94	0.88	0.85	0.86
#### Best Model: XGBoost outperformed other models across all key metrics, achieving an AUC-ROC of 0.94 and an F1-Score of 0.86. Given its superior precision and recall, it was deemed the most effective model for this task.

### 5. Key Fraud Predictors
#### The key variables driving fraud detection were:
#### Transaction Type: TRANSFER and CASH-OUT transactions are highly correlated with fraudulent activity.
#### Amount: Large transaction amounts, especially those exceeding a certain threshold, are common fraud indicators.
#### Balance Changes: Significant reductions in the originating account balance, particularly when funds are quickly transferred or withdrawn, are red flags.
#### Merchant Accounts: Unusual balance changes in accounts starting with "M" (merchant accounts) can suggest fraudulent behavior.

### 6. Validity of Predictors
#### These factors align with known fraud patterns:
#### High Transaction Values: Fraudsters often attempt to transfer large sums in a short period, aiming to deplete accounts before detection.
#### Targeted Transaction Types: Fraudulent activity is frequently concentrated in TRANSFER and CASH-OUT transactions, especially when significant funds are moved and rapidly withdrawn.
#### Rapid Balance Changes: Drastic shifts in account balances, particularly when unexpected, are telltale signs of fraud.

### 7. Recommended Fraud Prevention Measures
#### To safeguard the company’s infrastructure during updates, the following measures should be implemented:
#### Real-Time Fraud Detection: Integrate real-time monitoring to immediately flag suspicious high-value transactions.
#### Transaction Limits: Establish transaction limits for TRANSFER and CASH-OUT transactions, with multi-factor authentication required for any transaction exceeding the limit.
#### Behavioral Profiling: Use historical data to develop customer profiles based on typical transaction patterns. Deviations from expected behavior should trigger verification protocols.
#### Activity Alerts: Notify both the customer and internal security team when unusual activities are detected, enabling swift action.
#### Continuous Model Updates: Regularly update and retrain the fraud detection model to account for evolving fraud tactics.

### 8. Assessing the Effectiveness of Prevention Measures
#### To ensure these measures are effective, the following evaluation strategies are recommended:
#### Fraud Rate Comparison: Measure the incidence of flagged fraud before and after the implementation of new systems. A reduction would indicate success.
#### Performance Monitoring: Track performance metrics such as precision, recall, and false positive/negative rates. A higher precision without a corresponding drop in recall would signify improved detection accuracy.
#### A/B Testing: Apply the new measures to a sample group and leave the rest unchanged. A decrease in fraudulent activity within the test group would confirm the effectiveness of the measures.
#### Customer Feedback: Monitor customer complaints about unauthorized transactions. A reduction in complaints would reflect improved fraud detection.
#### Cost Analysis: Quantify financial savings resulting from reduced fraud incidents, which would demonstrate the impact of quicker detection.
#### By monitoring these key indicators, the company can continuously refine its fraud detection capabilities and safeguard its financial systems.